# Analiza orzeczeń sądów powszechnych

In [34]:
import spacy 
import pandas as pd
import numpy as np
import os
os.chdir('..')

In [37]:
import spacy
nlp = spacy.load('pl_core_news_lg')
dflist = []
for i in range(1,12):
    df = pd.read_csv('./resources/compensation/compensation_lemmatized_chunk_{}.zip'.format(i))
    dflist.append(df)
df = pd.concat(dflist)
del dflist
# df = df[['id', 'courttype', 'judgmenttype', 'textcontent']]
# df = df[df["courttype"] == 'COMMON'] 
# df = df[df["judgmenttype"] == 'SENTENCE']
# df['reasoning'] = df['text'].str.extract('(uzasadnienie[^>]*)', expand=True)
# df['sentence'] = df['text'].str.extract('([^>]*uzasadnienie)', expand=True)
# df['text'] = df.textcontent.str.replace(r'<[^<>]*>', '', regex=True)
# df.text = df.text.replace(r'\n',' ', regex=True) 
# df.text = df['text'].str.lower()
# df["lemmatized"] = df.reasoning.apply(lambda x: " ".join([y.lemma_ for y in nlp(str(x))]))

FileNotFoundError: [Errno 2] No such file or directory: './resources/compensation/compensation_lemmatized_chunk_1.zip'

In [ ]:
df['sentence'] = df['text'].str.extract('([^>]*uzasadnienie)', expand=True)
df.sentence = df.sentence.str.replace('\d\d\.\d\d\.\d\d\d\d','', regex=True)
df.sentence = df.sentence.str.replace('(?<=\d)\.(?=\d)','', regex=True)
df.sentence = df.sentence.str.replace('(\d)\s+(\d)','', regex=True)
df.sentence = df.sentence.str.replace('(\.\.\.)','', regex=True)
df['kwoty'] = df['sentence'].str.extract('(zasąd(?:za|ił)(?:\W+\w+){0,20}[\d+.]*)', expand=True)
df_kwoty = df[~df['kwoty'].isnull()]

In [ ]:
def findSum(str1):
 
    # A temporary string
    temp = "0"
 
    # holds sum of all numbers
    # present in the string
    Sum = 0
 
    # read each character in input string
    for ch in str1:
 
        # if current character is a digit
        if (ch.isdigit()):
            temp += ch
 
        # if current character is an alphabet
        else:
 
            # increment Sum by number found
            # earlier(if any)
            Sum += int(temp)
 
            # reset temporary string to empty
            temp = "0"
 
    # atoi(temp.c_str1()) takes care
    # of trailing numbers
    return Sum + int(temp)

In [ ]:
df_kwoty['target'] = df_kwoty['kwoty'].apply(lambda x: findSum(str(x)))
df_kwoty = df_kwoty[df_kwoty['target'] > 6000] 

<ipython-input-6-c307983ab50e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kwoty['target'] = df_kwoty['kwoty'].apply(lambda x: findSum(str(x)))


In [ ]:
# nlp = spacy.load('pl_core_news_lg')
# def get_polish_nouns(text):
#     # process the text with the Spacy model
#     doc = nlp(str(text))

#     # iterate over the tokens in the document
#     nouns = []
#     nouns = [token.text for token in doc if token.pos_ == "NOUN" and token.is_alpha and not token.is_digit and not token.ent_type_]

#     return nouns

In [ ]:
nlp = spacy.load('pl_core_news_lg')
def get_polish_words(text):
    # process the text with the Spacy model
    doc = nlp(str(text))

    # iterate over the tokens in the document
    words = []
    words = [token.text for token in doc if token.pos_ in ['NOUN', 'ADJ', 'VERB'] and token.is_alpha and not token.is_digit and not token.ent_type_ and len(token.text) > 2]

    return words

df_kwoty['words'] = df_kwoty['lemmatized'].apply(get_polish_words)

In [ ]:
q_hi  = df_kwoty["target"].quantile(0.99)
df_filtered = df_kwoty[(df_kwoty["target"] < q_hi)]

## Random Forest 

In [41]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
ylabels = df_filtered['target']
X = df_filtered['words'].astype(str)

stopwords = ['sąd','aca', 'lex','apelacja','apelacyjny','biegły', 'dokumentacja', 'adwokat', 'adwokacki', 'apelacja', 'wyrok', 'akta', 'protokolant','sędzia','ssa', 'spr','a','aby','ach','acz','aczkolwiek','aj','albo','ale','alez','ależ','ani','az','aż','bardziej','bardzo','beda','bedzie','bez','deda','będą','bede','będę','będzie','bo','bowiem','by','byc','być','byl','byla','byli','bylo','byly','był','była','było','były','bynajmniej','cala','cali','caly','cała','cały','ci','cie','ciebie','cię','co','cokolwiek','cos','coś','czasami','czasem','czemu','czy','czyli','daleko','dla','dlaczego','dlatego','do','dobrze','dokad','dokąd','dosc','dość','duzo','dużo','dwa','dwaj','dwie','dwoje','dzis','dzisiaj','dziś','gdy','gdyby','gdyz','gdyż','gdzie','gdziekolwiek','gdzies','gdzieś','go','i','ich','ile','im','inna','inne','inny','innych','iz','iż','ja','jak','jakas','jakaś','jakby','jaki','jakichs','jakichś','jakie','jakis','jakiś','jakiz','jakiż','jakkolwiek','jako','jakos','jakoś','ją','je','jeden','jedna','jednak','jednakze','jednakże','jedno','jego','jej','jemu','jesli','jest','jestem','jeszcze','jeśli','jezeli','jeżeli','juz','już','kazdy','każdy','kiedy','kilka','kims','kimś','kto','ktokolwiek','ktora','ktore','ktorego','ktorej','ktory','ktorych','ktorym','ktorzy','ktos','ktoś','która','które','którego','której','który','których','którym','którzy','ku','lat','lecz','lub','ma','mają','mało','mam','mi','miedzy','między','mimo','mna','mną','mnie','moga','mogą','moi','moim','moj','moja','moje','moze','mozliwe','mozna','może','możliwe','można','mój','mu','musi','my','na','nad','nam','nami','nas','nasi','nasz','nasza','nasze','naszego','naszych','natomiast','natychmiast','nawet','nia','nią','nic','nich','nie','niech','niego','niej','niemu','nigdy','nim','nimi','niz','niż','no','o','obok','od','około','on','ona','one','oni','ono','oraz','oto','owszem','pan','pana','pani','po','pod','podczas','pomimo','ponad','poniewaz','ponieważ','powinien','powinna','powinni','powinno','poza','prawie','przeciez','przecież','przed','przede','przedtem','przez','przy','roku','rowniez','również','sam','sama','są','sie','się','skad','skąd','soba','sobą','sobie','sposob','sposób','swoje','ta','tak','taka','taki','takie','takze','także','tam','te','tego','tej','ten','teraz','też','to','toba','tobą','tobie','totez','toteż','totobą','trzeba','tu','tutaj','twoi','twoim','twoj','twoja','twoje','twój','twym','ty','tych','tylko','tym','u','w','wam','wami','was','wasz','wasza','wasze','we','według','wiele','wielu','więc','więcej','wlasnie','właśnie','wszyscy','wszystkich','wszystkie','wszystkim','wszystko','wtedy','wy','z','za','zaden','zadna','zadne','zadnych','zapewne','zawsze','ze','zeby','zeznowu','zł','znow','znowu','znów','zostal','został','żaden','żadna','żadne','żadnych','że','żeby']

bow = TfidfVectorizer(
    stop_words=stopwords,
    ngram_range=(3, 3),
    min_df=30, max_df=700)

chi2_selector = SelectKBest(chi2, k=3000)


clf = RandomForestRegressor(random_state=0, min_samples_split=10)

pipel = Pipeline([('bow', bow),
                  ('chi2', chi2_selector),
                 ('classifier', clf)])

pipel.fit(X, ylabels)
print("Accuracy Score:",pipel.score(X, ylabels))
import eli5
features = eli5.show_weights(clf, vec=bow, top=1000)

Accuracy Score: 0.8232950369388309


In [42]:
features

Weight,Feature
0.1001 ± 0.1358,dowód przeprowadzić istnienie
0.0455 ± 0.1247,długotrwałość choroba rozmiar
0.0361 ± 0.0096,całość dać wiara
0.0327 ± 0.0537,data wyrokowania prowadzić
0.0292 ± 0.0399,fakt istotny sprawa
0.0140 ± 0.0336,dochodzić powódka zadośćuczynienie
0.0139 ± 0.0348,dowód niniejszy sprawa
0.0138 ± 0.0314,cywilny posiadacz kierować
0.0130 ± 0.0082,czas trwanie intensywność
0.0126 ± 0.0426,fizyczny psychiczny stopień


In [45]:
bow = TfidfVectorizer(
    stop_words=stopwords,
    ngram_range=(1, 1),
    min_df=30, max_df=700)

clf = LinearRegression(fit_intercept=True)

pipel = Pipeline([('bow', bow),
                  ('chi2', chi2_selector),
                 ('classifier', clf)])

pipel.fit(X, ylabels)
print("Accuracy Score:",pipel.score(X, ylabels))
import eli5
features = eli5.show_weights(clf, vec=bow, top=1000)

Accuracy Score: 0.9320228137871827


## Linear Regression

In [47]:
features

Weight?,Feature
+1270751.036,moczowy
+1234894.280,iniekcja
+1143298.109,nieuiszczone
+1074623.770,krowa
+1056026.299,czystość
+972602.005,domagania
+957673.496,poniesieniem
+957648.673,powiedzieć
+914961.064,krewny
+900956.737,konkurencja
